In [1]:
from pymilvus import Milvus, DataType


In [2]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility


In [7]:
# connections.connect()
connections.connect(
  alias="default", 
  host='localhost', 
  port='19530'
)


In [8]:
connections

In [6]:
TABLE_NAME = 'question_answering'

#Deleting previouslny stored table for clean run
if utility.has_collection(TABLE_NAME):
    collection = Collection(name=TABLE_NAME)
    collection.drop()

field1 = FieldSchema(name="id", dtype=DataType.INT64, descrition="int64", is_primary=True)
field2 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, descrition="float vector",dim=768, is_primary=False)
schema = CollectionSchema(fields=[field1, field2], description="collection description")
collection = Collection(name=TABLE_NAME, schema=schema)


KeyboardInterrupt: 

In [9]:
default_index = {"index_type": "IVF_FLAT", "metric_type": 'IP', "params": {"nlist": 200}}
collection.create_index(field_name="embedding", index_params=default_index)

KeyboardInterrupt: 

# transformer

In [7]:
# !pip install --upgrade pip
# !pip install transformers
# !pip install sentencepiece
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
# !pip install tqdm

In [20]:
import torch
print(torch.__version__)


1.11.0+cpu


In [8]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading: 100%|██████████| 129M/129M [00:11<00:00, 11.6MB/s] 
Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
text = "மனித உடலில் எத்தனை எலும்புகள் உள்ளன?"
tokens =tokenizer(text, truncation=True, padding="max_length", return_tensors="pt", max_length = 512)
tokens
res = model(**tokens)

In [21]:
res.last_hidden_state.shape

torch.Size([1, 512, 768])

## shrink the vector 

In [117]:
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings.tolist()


In [118]:
import pandas as pd

In [119]:
fp = r"tamil_qa.json"
with open(fp, "r") as read_file:
  qa = pd.read_json(read_file)

In [121]:
qa['id'] = [ i for i in range(len(qa))]

In [122]:
qa

,context,question,answer_text,answer_start,id
0,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,0
1,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,1
2,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,2
3,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,3
4,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,4
...,...,...,...,...,...
363,இந்திய ரூபாய் நாணயங்கள் (Coins of the Indian r...,இந்திய நாணய சட்டம் எந்த ஆண்டு இயற்றப்பட்டது?,1955 ஆம் ஆண்டு செப்டம்பர்,5154,363
364,"ஜெர்மனி (Germany, [ˈdʒɜːmənɪ]), அல்லது ஜெர்மன்...",ஜெர்மனியில் மிகப்பெரிய மதம் எது?,கிறித்தவத்தை,30737,364
365,"கெய்ரோ (Cairo, அரபு மொழியில்: القاهرة - அல்-கா...",கெய்ரோ நகரத்தின் பரப்பளவு என்ன?,453 square kilometers,1507,365
366,வாழைப்பழம் (banana) என்பது தாவரவியலில் சதைப்பற...,மிகவும் அரிதான வாழைப்பழம் என்றால் என்ன?,சிங்கன்,7688,366


In [134]:
contexts = qa['context'].to_list()
ids = qa['id'].to_list()


In [135]:
size = 10

In [ ]:
res = encode(contexts[:size])

In [136]:
"""
[[], [[]]]
"""
data = [ids[:size], res]

In [133]:
len(res)

10

In [146]:
mr = collection.insert(data)
mr

(insert count: 10, delete count: 0, upsert count: 0, timestamp: 432815049325150211)

In [138]:
collection.load()

In [109]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

In [142]:
results = collection.search(
	data=[data[1][1]], 
	anns_field="embedding", 
	param=search_params, 
	limit=10, 
	expr=None,
	consistency_level="Strong"
)

In [143]:
for r in results:
    print(r)

['(distance: 0.0, id: 1)', '(distance: 0.06303973495960236, id: 6)', '(distance: 0.06418201327323914, id: 8)', '(distance: 0.14161071181297302, id: 9)', '(distance: 0.16504788398742676, id: 2)', '(distance: 0.20349864661693573, id: 7)', '(distance: 0.22696754336357117, id: 4)', '(distance: 0.242176815867424, id: 3)', '(distance: 0.353226900100708, id: 5)', '(distance: 0.3694557249546051, id: 0)']


In [107]:
# query
res = collection.query(
  expr = "id <2", 
  output_fields = ["id"],
  consistency_level="Strong"
)

In [113]:
print(res)

[{'id': 1}]
